In [8]:
from matplotlib.axis import YAxis
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import pickle

In [9]:
#Import Scraped rib.gg Data
df = pd.read_csv("../Amer-VCT-Spring2023/data/EventEssentials+Players.csv")

#Can import your own training or testing data.

In [10]:
#Set Attack Team Win as target variable
df['ATKWin'] = 0
for i in range(0, len(df)):

    if(df.loc[i, 'attackingTeamNumber'] == df.loc[i, 'winningTeamNumber']):
        df.loc[i, 'ATKWin'] = 1

In [11]:
#Seperate Independent and Dependent Variables
indVars = df.drop(columns=['roundId_x', 'attackingTeamNumber', 'winningTeamNumber','ATKWin', 'playerId', 'assistants', 'victimId'])
depVar = df[['matchId_y', 'ATKWin']]

In [12]:
#Load log reg model or skip to create new model. Can skip to the cell that creates testing set.
with open('impactModel.pkl', 'rb') as f:
    logreg = pickle.load(f)

In [ ]:
start_test_matchId = 112510

#Train on group stage
X_train = indVars[indVars['matchId_y'] < start_test_matchId].drop(columns=['matchId_y'])
Y_train = depVar[depVar['matchId_y'] < start_test_matchId].drop(columns=['matchId_y'])


In [ ]:
#do Logisistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train.values.ravel())

In [ ]:
#Save the log reg model for later use
with open('impactModel.pkl','wb') as f:
    pickle.dump(logreg, f)

In [14]:
start_test_matchId = 112510

#Test on playoffs
X_test = indVars[indVars['matchId_y'] >= start_test_matchId].drop(columns=['matchId_y'])
X_test_MatchID = indVars[indVars['matchId_y'] >= start_test_matchId].drop_duplicates(subset=['matchId_y'])
y_test = depVar[depVar['matchId_y'] >= start_test_matchId].drop(columns=['matchId_y'])

In [15]:
#Get probability of [ATKWin=0, ATKWin=1]
y_pred=logreg.predict_proba(X_test)

print(len(y_pred))

5383


In [16]:
#Transform Bomb Time from (100000 to -450000) -> (0 to 1)
def NormalizeData(data):
    # return (data - np.max(data)) / (np.min(data) - np.max(data))
    return (data - np.max(data)) / (-45000 - np.max(data))

In [17]:
#get list of halftimes
halftimes = {}

for i in X_test_MatchID.matchId_y:

    #Set X-axis for graph (round + event time)
    xAx = df[df['matchId_y'] == i]['roundId_x'] - df[df['matchId_y'] == i]['roundId_x'][df[df['matchId_y'] == i].index[0]] + 1 + NormalizeData(df[df['matchId_y'] == i]['roundTime'])

    #Find index of round 13
    halftime = xAx.loc[xAx==13].index[0] 
    firstRound = xAx.loc[xAx==1].index[0] 
    lastRound = xAx.last_valid_index()

    halftimes[i] = [firstRound, halftime, lastRound]


print(halftimes)
# print(df.iloc[2354:2366])



{114166: [0, 108, 213], 114165: [214, 320, 436], 114164: [437, 541, 618], 114172: [619, 732, 778], 114171: [779, 887, 988], 114170: [989, 1088, 1163], 114169: [1164, 1264, 1364], 114174: [1365, 1483, 1605], 113981: [1606, 1712, 1799], 113980: [1800, 1899, 1993], 113507: [1994, 2103, 2144], 113506: [2145, 2246, 2358], 113505: [2359, 2469, 2568], 113166: [2569, 2678, 2724], 113165: [2725, 2839, 2936], 113164: [2937, 3040, 3116], 113168: [3117, 3221, 3308], 113167: [3309, 3418, 3493], 112520: [3494, 3598, 3620], 112519: [3621, 3728, 3778], 112517: [3779, 3882, 3930], 112516: [3931, 4030, 4079], 112515: [4080, 4185, 4253], 112514: [4254, 4356, 4503], 112513: [4504, 4608, 4712], 112512: [4713, 4820, 4888], 112511: [4889, 4996, 5236], 112510: [5237, 5336, 5382]}


In [18]:
#Get probabilities for home team
yAx = []
for i in X_test_MatchID.matchId_y:

    firstRound = halftimes[i][0]
    halftime = halftimes[i][1]
    lastRound = halftimes[i][2]

    #Match prediction with the respective sides
    if(df[df['matchId_y'] == i].iloc[0]['attackingTeamNumber'] == 1):
        firstHalf = [item[1] for item in y_pred[firstRound:halftime]]
        secHalf = [item[0] for item in y_pred[halftime:lastRound+1]]
    else:
        firstHalf = [item[0] for item in y_pred[firstRound:halftime]]
        secHalf = [item[1] for item in y_pred[halftime:lastRound+1]]

    #Set y-axis for graph (Win probs)
    full = firstHalf + secHalf
    yAx += full


# print(yAx[0:30])
print(len(yAx))


5383


In [19]:
#Create impact df
playerImpact = df[['matchId_y', 'roundId_x', 'attackingTeamNumber', 'playerId', 'assistants', 'victimId']]
playerImpact = playerImpact[playerImpact['matchId_y'] >= start_test_matchId]
playerImpact['impact'] = 0

print(len(playerImpact))


5383


In [20]:
#Calculate impact for each player
for i in range(0, len(playerImpact)):
    
    matchId = playerImpact.loc[playerImpact.index[0]+i, 'matchId_y']
    playerId = playerImpact.loc[playerImpact.index[0]+i, 'playerId'] 

    #Impact is change in win prob
    if(not math.isnan(playerId)):
        playerImpact.loc[playerImpact.index[0]+i, 'impact'] = abs(yAx[i]-yAx[i-1])
    
    else:
        pass

In [21]:
#Get player info (Used rib.gg Discord bot to get data)
playerNames = pd.read_csv("../Amer-VCT-Spring2023/data/AllPlayers.csv")

In [22]:
#gVR = Victim impact by round, gVM = Victim impact by match 
gVR = playerImpact.groupby(['roundId_x', 'victimId'])['impact'].sum().reset_index()
gVM = playerImpact.groupby(['victimId'])['impact'].sum().reset_index()

print(gVM.head(10))

   victimId     impact
0      81.0  15.822778
1     165.0  29.458823
2     440.0  11.425130
3    1023.0  24.174025
4    1506.0  15.768590
5    2423.0  13.615445
6    2483.0  29.816493
7    2749.0  12.144943
8    2752.0  13.321170
9    2841.0  29.858479


In [23]:
#Getting impact for assistants
playerImpact['impact_assist'] = 0

for i in range(0, len(playerImpact)):

    if(len(json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])) > 0):

        #Split total impact between all assistants and main player
        playerImpact.loc[playerImpact.index[0]+i,'impact_assist'] = playerImpact.loc[playerImpact.index[0] + i, 'impact'] * (0.2 / len(json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])))
        playerImpact.loc[playerImpact.index[0]+i,'impact'] -= playerImpact.loc[playerImpact.index[0] + i, 'impact'] * (0.2)

In [24]:
#Create assistant impact dataframe
ass = pd.DataFrame(pd.np.empty((0, 8)))
ass.columns = playerImpact.columns.tolist()

for i in range(0, len(playerImpact)):

    for p in (json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])):

        # print(p)
        row =  pd.DataFrame([{'matchId_y': playerImpact.loc[playerImpact.index[0]+i,'matchId_y'],
         'roundId_x': playerImpact.loc[playerImpact.index[0]+i,'roundId_x'], 
         'attackingTeamNumber': playerImpact.loc[playerImpact.index[0]+i,'attackingTeamNumber'],
         'playerId': p,
         'assistants': [],
         'victimId': playerImpact.loc[playerImpact.index[0]+i,'victimId'],
         'impact': playerImpact.loc[playerImpact.index[0]+i,'impact_assist']}])
        
        ass = pd.concat([ass, row])

        # print(row)

ass['impact_assist'] = 0


C:\Users\Renzjordan\AppData\Local\Temp\ipykernel_12908\1285282499.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  ass = pd.DataFrame(pd.np.empty((0, 8)))


In [25]:
#Impact in playoffs

#gA = Total impact with assists
gA = ass.groupby(['playerId'])['impact'].sum().reset_index()

#gP = Total impact as main player
gP = playerImpact.groupby(['playerId'])['impact'].sum().reset_index()

#Total impact = Main impact + Assist Impact - Victim Impact
gP['impact'] = gP['impact'] + gA['impact'] - gVM['impact']

#Add player names to the total impact df
gP = pd.merge(gP, playerNames, how='left', left_on = 'playerId', right_on = 'Player Id')


print(gP.sort_values('impact', ascending=False).to_markdown())

|    |   playerId |     impact |   Player Id | Ign       | Team          |
|---:|-----------:|-----------:|------------:|:----------|:--------------|
|  1 |        165 |  6.06511   |         165 | s0m       | NRG Esports   |
| 18 |       4746 |  5.91365   |        4746 | Less      | LOUD          |
|  2 |        440 |  5.2216    |         440 | Zellsis   | Cloud9        |
| 24 |      10253 |  4.64193   |       10253 | Ethan     | Evil Geniuses |
| 23 |       6414 |  4.32535   |        6414 | kiNgg     | Leviatán      |
|  6 |       2483 |  3.75306   |        2483 | crashies  | NRG Esports   |
| 29 |      23845 |  3.32429   |       23845 | Demon1    | Evil Geniuses |
|  7 |       2749 |  2.40433   |        2749 | Khalil    | FURIA         |
| 11 |       2895 |  2.21063   |        2895 | Saadhak   | LOUD          |
| 10 |       2882 |  1.96406   |        2882 | C0M       | Evil Geniuses |
| 25 |      13591 |  1.8521    |       13591 | Xeppaa    | Cloud9        |
|  5 |       2423 |  1.33